In [1]:
%matplotlib inline
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
import re
import seaborn as sns

import pandas as pd
import numpy as np
import datetime
import pandas as pd
import timeit

from sklearn.neural_network import MLPRegressor 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df_airquality = pd.DataFrame.from_csv('data/airquality.csv')
df_district = pd.DataFrame.from_csv('data/district.csv')
df_meteorology = pd.DataFrame.from_csv('data/meteorology.csv')
df_station = pd.DataFrame.from_csv('data/station.csv')
df_weatherforecast = pd.DataFrame.from_csv('data/weatherforecast.csv')

In [3]:
final_df = df_airquality.join(df_station)
final_df = final_df.reset_index()
final_df['station_id'] = final_df['\ufeffstation_id']
final_df = final_df.drop('\ufeffstation_id',1)
final_df = final_df.join(df_district, on=['district_id'],rsuffix='_district')

In [4]:
df_city = pd.DataFrame.from_csv('data/city.csv')
df_city = df_city.reset_index()

df_city['city_id'] = df_city['\ufeffcity_id']
df_city['latitude_city']= df_city['longitude']
df_city['longitude_city']= df_city['longitude']

df_city = df_city.drop('latitude',1)
df_city = df_city.drop('longitude',1)

df_city = df_city.drop('\ufeffcity_id',1)
df_city['city_id'] = df_city.city_id.apply(lambda x : int(x))
final_df = pd.merge(final_df, right=df_city,on=['city_id'],how='left')

In [5]:
final_df = final_df.drop('name_chinese_x',1)
final_df = final_df.drop('name_chinese_district',1)
final_df = final_df.drop('name_chinese_y',1)
final_df = final_df.drop('name_english_y',1)

In [6]:
df_meteorology = df_meteorology.reset_index()
df_meteorology['id'] = df_meteorology['\ufeffid']
df_meteorology = df_meteorology.drop('\ufeffid',1)


In [7]:
df_meteorology['district_id'] = df_meteorology['id']
df_meteorology = df_meteorology.drop('id',1)
final_df = pd.merge(final_df, right=df_meteorology,on=['district_id','time'],how='outer')

In [8]:
final_df['latitude_city']  = final_df['latitude_city'].apply(lambda x : str(x))
final_df['longitude_city']  = final_df['longitude_city'].apply(lambda x : str(x))

final_df['latitude']  = final_df['latitude'].apply(lambda x : str(x))
final_df['longitude']  = final_df['longitude'].apply(lambda x : str(x))

final_df['lat-lon-city'] = final_df['latitude_city'] + ' ' + final_df['longitude_city']
final_df['lat-lon-district'] = final_df['latitude'] + ' ' + final_df['longitude']

final_df = final_df.drop('latitude_city',1)
final_df = final_df.drop('longitude_city',1)

final_df = final_df.drop('latitude',1)
final_df = final_df.drop('longitude',1)

In [9]:
final_df['time'] = final_df.time.apply(lambda x : datetime.datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S'))

final_df['year']    = final_df['time'].apply(lambda x : x.year)
final_df['month']   = final_df['time'].apply(lambda x : x.month)
final_df['day']     = final_df['time'].apply(lambda x : x.day)
final_df['hour']    = final_df['time'].apply(lambda x : x.hour)

final_df = final_df.drop('time',1)

In [10]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3923229 entries, 0 to 3923228
Data columns (total 24 columns):
PM25_Concentration       float64
PM10_Concentration       float64
NO2_Concentration        float64
CO_Concentration         float64
O3_Concentration         float64
SO2_Concentration        float64
name_english_x           object
district_id              float64
station_id               float64
name_english_district    object
city_id                  float64
cluster_id               float64
weather                  float64
temperature              float64
pressure                 float64
humidity                 float64
wind_speed               float64
wind_direction           float64
lat-lon-city             object
lat-lon-district         object
year                     int64
month                    int64
day                      int64
hour                     int64
dtypes: float64(16), int64(4), object(4)
memory usage: 748.3+ MB


In [11]:
# Save it to a temporary data frame 
final_df.to_csv('data/final_dataframe.csv')

In [12]:
# final_df = pd.DataFrame.from_csv('data/final_dataframe.csv')

In [13]:
final_df.columns

Index(['PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration',
       'CO_Concentration', 'O3_Concentration', 'SO2_Concentration',
       'name_english_x', 'district_id', 'station_id', 'name_english_district',
       'city_id', 'cluster_id', 'weather', 'temperature', 'pressure',
       'humidity', 'wind_speed', 'wind_direction', 'lat-lon-city',
       'lat-lon-district', 'year', 'month', 'day', 'hour'],
      dtype='object')

In [14]:
final_df.head()

PM25_Concentration  PM10_Concentration  NO2_Concentration  \
0               138.0               159.4               56.3   
1                89.0               132.9               30.5   
2                87.0               165.3               78.9   
3                91.0               162.2               97.0   
4               124.0               163.9               38.7   

   CO_Concentration  O3_Concentration  SO2_Concentration  \
0               0.9              50.8               17.2   
1               0.8              96.5                7.6   
2               0.8              67.4               16.3   
3               0.9              58.6               28.6   
4               0.9              51.1               17.9   

            name_english_x  district_id  station_id name_english_district  \
0        HaiDianBeiBuXinQu        101.0      1001.0             HaiDianQu   
1  HaiDianBeiJingZhiWuYuan        101.0      1002.0             HaiDianQu   
2            HaiDianWanLiu        101.0      1006.0             HaiDianQu   
3         XiZhiMenBeiDaJie        101.0      1008.0             HaiDianQu   
4        HaiDianBeiBuXinQu        101.0      1001.0             HaiDianQu   

  ...   pressure  humidity  wind_speed  wind_direction           lat-lon-city  \
0 ...     1004.0      56.0        7.92            13.0  116.407394 116.407394   
1 ...     1004.0      56.0        7.92            13.0  116.407394 116.407394   
2 ...     1004.0      56.0        7.92            13.0  116.407394 116.407394   
3 ...     1004.0      56.0        7.92            13.0  116.407394 116.407394   
4 ...     1004.0      64.0        7.56            13.0  116.407394 116.407394   

                lat-lon-district  year  month day hour  
0  40.090678999999994 116.173553  2014      5   1    0  
1    40.00395 116.20531000000001  2014      5   1    0  
2           39.987313 116.287451  2014      5   1    0  
3  39.954046999999996 116.348991  2014      5   1    0  
4  40.090678999999994 116.173553  2014      5   1    1  

[5 rows x 24 columns]

In [15]:
train_x_numerical = [ 'year', 'month', 'day', 'hour' , 'district_id', 'city_id']
train_x_string = ['name_english_x', 'lat-lon-city', 'lat-lon-district',  'name_english_district', ]           
train_x_weather = ['weather', 'temperature', 'pressure', 'humidity','wind_speed', 'wind_direction']

train_columns = []
train_columns.extend(train_x_numerical)
train_columns.extend(train_x_weather)

train_y_pollution = ['PM25_Concentration', 'PM10_Concentration', 'NO2_Concentration', 'CO_Concentration', 'O3_Concentration', 'SO2_Concentration']


In [16]:
df_null = final_df[train_columns][final_df[train_columns].isnull().any(axis=1)]
df_null.head(10)

year  month  day  hour  district_id  city_id  weather  temperature  \
567   2014      5    8    23        101.0      1.0      NaN         17.0   
568   2014      5    8    23        101.0      1.0      NaN         17.0   
569   2014      5    8    23        101.0      1.0      NaN         17.0   
570   2014      5    8    23        101.0      1.0      NaN         17.0   
767   2014      5   11     1        101.0      1.0      5.0          NaN   
768   2014      5   11     1        101.0      1.0      5.0          NaN   
769   2014      5   11     1        101.0      1.0      5.0          NaN   
770   2014      5   11     1        101.0      1.0      5.0          NaN   
2402  2014      5   28     3        101.0      1.0      0.0         18.0   
2403  2014      5   28     3        101.0      1.0      0.0         18.0   

      pressure  humidity  wind_speed  wind_direction  
567     1010.0       0.0        5.76            13.0  
568     1010.0       0.0        5.76            13.0  
569     1010.0       0.0        5.76            13.0  
570     1010.0       0.0        5.76            13.0  
767        NaN       NaN         NaN            14.0  
768        NaN       NaN         NaN            14.0  
769        NaN       NaN         NaN            14.0  
770        NaN       NaN         NaN            14.0  
2402     998.0       0.0         NaN            23.0  
2403     998.0       0.0         NaN            23.0

## Trying simple logistic regression for district predictons 

1. Type 1 fill NaN with 0 to see how the things are performing 
2. Remove rows with NaN to see what's the impact 

In [17]:
df = final_df.fillna(0.0)
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

3138583 784646

Coefficients: 
 [[  1.55762175e+01   1.95342076e+00   3.61902924e-01  -4.08455589e-02
   -3.00181969e-03   4.02630014e-01   6.05173368e+00  -1.25332361e+00
    1.05508650e-02  -1.28625801e-01  -1.24342638e+00   3.38018422e-01]
 [  3.11785805e+01   3.19151036e+00   8.54310039e-01   1.48112723e-01
   -4.37922862e-03   6.83271037e-01   6.13580772e+00  -1.72573934e+00
    1.82745120e-02  -4.01613308e-01  -1.26196311e+00   5.83732129e-01]
 [  1.08787849e+01   1.18864764e+00   1.25169012e-01   8.22818674e-02
   -1.72090750e-03   2.00495632e-01   1.60837738e+00  -4.45439653e-01
    6.26399725e-03  -1.39566818e-01  -8.54402669e-01   2.78894821e-02]
 [  2.72058864e-01   3.21491087e-02   3.54835191e-03  -2.03398816e-03
   -5.31035231e-05   8.47634838e-03   7.30594100e-02  -2.54876667e-02
    1.93238661e-04  -1.48823164e-03  -2.59591811e-02   5.22802501e-03]
 [ -2.54852732e+01  -3.16393803e+00   1.04676432e-01   1.19299399e+00
   -1.72930895e-03   2.37901891e-01  -6.10994197e-02  

In [18]:
# Removing entries with NaN values in the regression model
# Helped to impove scores
df = final_df[~final_df.isnull().any(axis=1)]
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

620164 155042

Coefficients: 
 [[ -3.13631059e+01  -2.58823606e+00   6.51987002e-01   4.15731465e-01
    1.92387084e+00  -1.92187673e+02   7.27827135e+00  -1.93888325e+00
    4.64978324e-01   2.54938670e-01  -1.79439918e+00   6.65270819e-01]
 [ -3.07091207e+01  -2.68853257e+00   1.61549086e+00   4.97979102e-01
    3.33891188e+00  -3.33560400e+02   9.41107469e+00  -2.30279762e+00
    5.74858671e-01  -1.19701773e-01  -1.51332596e+00   1.05024446e+00]
 [ -1.90927675e+01  -1.82316692e+00   1.74845515e-01   2.76665832e-01
    3.01882933e-01  -3.01096139e+01   1.50057811e+00  -7.93003154e-01
    2.49773999e-01   5.78236213e-02  -1.29612034e+00   1.16374588e-01]
 [ -7.02896832e-01  -6.71736398e-02   7.46311573e-03   2.25744082e-03
    1.64354924e-02  -1.63910846e+00   8.89120612e-02  -4.28565084e-02
    8.44715307e-03   3.65403677e-03  -3.70146230e-02   9.18426531e-03]
 [ -2.48547631e-01  -6.35947670e-01   8.35901572e-02   1.02029264e+00
   -2.03622174e-02   2.01355880e+00  -2.72073244e-01   

#### Using averages to estimate values and rerunning the regressions on the dataframe

In [19]:
merged_df = final_df.groupby(train_x_numerical).mean().reset_index()
print ("With remove NaN's ", len(final_df[~final_df.isnull().any(axis=1)]))
print ("Without removed Nan's " , len(final_df))
print ("Length of DF with merged ", len(merged_df[~merged_df.isnull().any(axis=1)]))

With remove NaN's  775206
Without removed Nan's  3923229
Length of DF with merged  615432


In [ ]:
# Removing entries with NaN values in the regression model
# Helped to impove scores
df = merged_df[~merged_df.isnull().any(axis=1)].fillna(0.0)
train_df, test_df = train_test_split(df, test_size = 0.2)
print(len(train_df) , len(test_df))

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_df[train_columns], train_df[train_y_pollution])

# The coefficients
print('\nCoefficients: \n', regr.coef_)
# The mean squared error
print("\n\nMean squared error for " )
print(np.mean((regr.predict(test_df[train_columns]) - test_df[train_y_pollution]) ** 2))
# Explained variance score: 1 is perfect prediction
print('\nVariance score: %.2f' % regr.score(test_df[train_columns], test_df[train_y_pollution]))

492345 123087

Coefficients: 
 [[ -2.34720166e+01  -1.85358503e+00   6.69236614e-01   5.00045319e-01
    1.98814226e+00  -1.98632152e+02   7.51921154e+00  -1.88465673e+00
    4.81001362e-01   3.45029466e-01  -1.93290811e+00   6.93165372e-01]
 [ -1.36359616e+01  -9.84182529e-01   1.69317317e+00   5.80596590e-01
    3.41563156e+00  -3.41245441e+02   9.70411918e+00  -2.13872590e+00
    6.04701097e-01   8.85023101e-03  -1.69707222e+00   1.08451913e+00]
 [ -1.44276199e+01  -1.34097990e+00   1.74013692e-01   3.00520678e-01
    3.31642651e-01  -3.30880200e+01   1.51417890e+00  -7.51069804e-01
    2.44785452e-01   7.67495512e-02  -1.29168628e+00   9.31394581e-02]
 [ -6.70509605e-01  -6.56058163e-02   8.21627117e-03   2.10936181e-03
    1.52391638e-02  -1.51961396e+00   9.21616445e-02  -4.40863313e-02
    8.50710211e-03   3.69870907e-03  -3.93850432e-02   8.74141743e-03]
 [ -1.33604936e+00  -8.62172348e-01   4.63913904e-02   9.86585070e-01
   -6.41941233e-02   6.40788682e+00  -2.50200245e-01   

### Using a more complicated neural network based approach

Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. For example, scale each attribute on the input vector X to `[0, 1]` or `[-1, +1]`, or standardize it to have mean `0` and variance `1`. Note that you must apply the same scaling to the test set for meaningful results. You can use StandardScaler for standardization.


In [ ]:
merged_df = final_df.groupby(train_x_numerical).mean().reset_index()
mask = ~merged_df.isnull().any(axis=1)
df = merged_df[mask]
train_df, test_df = train_test_split(df, test_size = 0.2)

X_train = train_df[train_columns]
Y_train = train_df[train_y_pollution]

X_test = train_df[train_columns]
Y_test = train_df[train_y_pollution]

scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

regr = MLPRegressor()

print('Train / Test ', len(train_df) , len(test_df))
print('Training neural Network model might take some time')
start = timeit.default_timer()
regr.fit( X_train , Y_train)
stop = timeit.default_timer()
train_time = int(stop - start)
print('Training finshed in ',train_time , 'seconds')
print("\n\nMean squared error for ")
print(np.mean((regr.predict(X_test) - Y_test) ** 2))
print('\nVariance score: %.2f' % regr.score(X_test, Y_test))

Train / Test  492345 123087
Training neural Network model might take some time


In [ ]:
# With more parmeters for processing 

regr = MLPRegressor(hidden_layer_sizes=(200, ) , max_iter=500)

print('Train / Test ', len(train_df) , len(test_df))
print('Training neural Network model might take some time')
start = timeit.default_timer()
regr.fit( X_train , Y_train)
stop = timeit.default_timer()
train_time = int(stop - start)
print('Training finshed in ',train_time , 'seconds')
print("\n\nMean squared error for ")
print(np.mean((regr.predict(X_test) - Y_test) ** 2))
print('\nVariance score: %.2f' % regr.score(X_test, Y_test))

We can see, using a simple neural network based classifier we can improve on the variance by a lot.

### Running for all the data without merging the data.

In [ ]:
df = final_df.fillna(-99)
train_df, test_df = train_test_split(df, test_size = 0.2)

print(len(train_df) , len(test_df))

X_train = train_df[train_columns]
Y_train = train_df[train_y_pollution]

X_test = train_df[train_columns]
Y_test = train_df[train_y_pollution]

scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

regr = MLPRegressor(hidden_layer_sizes=(200, ) , max_iter=500)

print(len(train_df) , len(test_df))
print('Training neural Network model might take some time')
start = timeit.default_timer()
regr.fit( X_train , Y_train)
stop = timeit.default_timer()
train_time = int(stop - start)
print('Training finshed in ',train_time , 'seconds')
print("\n\nMean squared error for ")
print(np.mean((regr.predict(X_test) - Y_test) ** 2))
print('\nVariance score: %.2f' % regr.score(X_test, Y_test))